In [1]:
import torch
import torch.nn as nn
import numpy as np

# 权值初始化

网络模型搭建后，对网络中权重进行合适的初始化非常重要。
- 初始化正好在模型的最优解附近，则模型训练速度会非常快。
- 初始化离最优解远，则模型需要更多次迭代，可能会引发梯度消失或爆炸。

## 梯度消失和爆炸

<img style="float: center;" src="images/79.png" width="70%">

要计算$W_2$梯度，根据链式法则：$H_2=H_1\times W_2$

$\triangle W_2=\frac{\partial Loss}{\partial W_2}=\frac{\partial Loss}{\partial out}\times \frac{\partial out}{\partial H_2}\times \frac{\partial H_2}{\partial W_2}=\frac{\partial Loss}{\partial out}\times \frac{\partial out}{\partial H_2}\times H_1$

可以发现$W_2$梯度求解过程中会用到上一层神经元输出值$H_1$，若$H_1$的输出值非常小，则$W_2$的梯度也会非常小（梯度消失），尤其当网络层非常多的时候，连乘一个非常小的数，会导致越乘越小。而当$H_1$非常大的时候（梯度爆炸）。

$H_1 \rightarrow 0 \Rightarrow \triangle W_2 \rightarrow 0$

$H_1 \rightarrow \infty \Rightarrow \triangle W_2 \rightarrow \infty$

一旦发生梯度消失或爆炸，模型就无法训练，要想避免这个现象，需要**控制网络输出层的尺度范围，不能让它太大或太小**，这就需要合理的**初始化权重**。

In [2]:
# 建立100层多层感知机，每一层256个神经元
class MLP(nn.Module):
    def __init__(self, neural_num, layers):
        super(MLP, self).__init__()
        self.linears = nn.ModuleList([nn.Linear(neural_num, neural_num, bias=False)
                                     for i in range(layers)])
        self.neural_num = neural_num
    
    # 正向传播
    def forward(self, x):
        for i, linear in enumerate(self.linears):
            x = linear(x)
        
            print(f"layer:{i},std:{x.std()}")

            if torch.isnan(x.std()):
                print(f"output is nan in {i} layers")
                break
        
        return x
    
    # 权值初始化，使用标准正态分布
    def initialize(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight.data)

In [3]:
layer_nums = 100
neural_nums = 256
batch_size = 16

net = MLP(neural_nums, layer_nums)
net.initialize()

inputs = torch.randn((batch_size, neural_nums))

output = net(inputs)
output

layer:0,std:15.944011688232422
layer:1,std:255.11569213867188
layer:2,std:4016.758544921875
layer:3,std:64151.5234375
layer:4,std:1039510.125
layer:5,std:16374433.0
layer:6,std:263322816.0
layer:7,std:4190569728.0
layer:8,std:67863425024.0
layer:9,std:1070481801216.0
layer:10,std:16919611572224.0
layer:11,std:275770756825088.0
layer:12,std:4517903479078912.0
layer:13,std:7.369505890946253e+16
layer:14,std:1.1669860450324972e+18
layer:15,std:1.821879882059717e+19
layer:16,std:2.91014902543194e+20
layer:17,std:4.665675452235282e+21
layer:18,std:7.621574055228135e+22
layer:19,std:1.211707098629305e+24
layer:20,std:1.9111977476024693e+25
layer:21,std:3.043657431929855e+26
layer:22,std:4.9322685894834766e+27
layer:23,std:7.822334209640582e+28
layer:24,std:1.2538825974158207e+30
layer:25,std:2.0024991819955308e+31
layer:26,std:3.1725463099290994e+32
layer:27,std:5.203591204483248e+33
layer:28,std:8.078508999111262e+34
layer:29,std:1.3377619780819773e+36
layer:30,std:nan
output is nan in 30 l

tensor([[ 1.8903e+37, -2.4083e+36,  4.2697e+37,  ...,  6.3164e+37,
          3.7192e+37, -2.8457e+37],
        [-1.3344e+36, -3.5256e+37,  2.1467e+36,  ...,  2.5005e+37,
          1.8009e+37, -4.6022e+37],
        [-1.8832e+36,  1.3759e+37,  2.7584e+35,  ...,  1.0284e+37,
          1.6108e+37,  6.3711e+35],
        ...,
        [-1.1908e+37,  5.5315e+36, -9.7927e+36,  ..., -5.2890e+36,
         -2.2129e+37,  8.1288e+36],
        [ 1.8008e+36,  4.1029e+36,  1.8122e+37,  ...,  3.9453e+37,
         -1.4130e+37, -1.4482e+36],
        [-1.8667e+37,  9.6412e+36, -1.7507e+36,  ..., -1.3098e+37,
          1.6047e+37,  1.5941e+37]], grad_fn=<MmBackward0>)

在30层左右，网络的输出成为nan，导致后续输出值过大。

如果进行反向传播，根据上面的权重推导公式，当值为nan，反向传播时这些权重无法进行更新，会发生梯度爆炸现象。

有时候在训练网络时，**最后结果全是nan的原因，可能就是权重初始化不当导致的**。

**为啥权重初始化不当会影响网络输出？**

可以推导$D(XY)$方差公式：

借助三个基本公式：
- $E(XY)=E(X)E(Y)$
- $D(X)=E(X^2)-[E(X)]^2$
- $D(X+Y)=D(X)+D(Y)$

则：

$D(XY)$

$=E[XY-E(XY)]^2$

$=E(X^2Y^2)-2XYE(XY)+E^2(XY)$

$=E(X^2)E(Y^2)-2E^2(X)E^2(Y)+E^2(X)E^2(Y)$

$=E(X^2)E(Y^2)-E^2(X)E^2(Y)$

$=(D(X)+[E(X)]^2)(D(Y)+[E(Y)]^2)-E(X^2)E(Y^2)-E^2(X)E^2(Y)$

$=D(X)D(Y)+D(X)[E(Y)]^2+D(Y)[E(X)]^2$

若$E(X)=0,E(Y)=0,则D(XY)=D(X)D(Y)$

则以下网络：
<img style="float: center;" src="images/79.png" width="70%">

第一层第一个神经元方差：

$H_{11}=\sum^n_{i=0}X_i*W_{1i}$

$D(H_{11})=\sum^n_{i=0}D(X_i)*D(W_{1i})=n*(1*1)=n$

$std(H_{11})=\sqrt{D(H_{11})}=\sqrt(n)$

此处，输入数据的权重都初始化为均值为0，方差为1的标准正态，经过一个网络层发现方差扩大了n倍。（上面用了100个网络层，则方差会指数增长，因此出现输出层方差为nan的情况）

**如何解决呢？**

让网络层的输出方差保持尺度不变即可。

根据公式，$D(H_{11})=\sum^n_{i=0}D(X_i)*D(W_{1i})=n*(1*1)=n$，发现每一层的输出方差与每一层神经元个数、前一层输出方差、本层权重有关，如果想让方差尺度不变，可以让每一层输出方差为1，即$D(H_{11})=1$，这样后面多层相乘，尺度不变。

由于神经元个数无法改变，前一层的输出方差是1，因此只能改变本层权重方差：

$D(H_1)=n\times D(X)\times D(W)=1$

$D(W)=\frac{1}{n} \Rightarrow std(W)=\sqrt{\frac{1}{n}}$

即，在权重初始化时，方差如果为$\sqrt{1}{n}$的话，每一层的输入方差都是1，这样方差就不会导致nan的情况发生。

In [4]:
class MLP(nn.Module):
    def __init__(self, neural_num, layers):
        super(MLP, self).__init__()
        self.linears = nn.ModuleList([nn.Linear(neural_num, neural_num, bias=False)
                                     for i in range(layers)])
        self.neural_num = neural_num
    
    # 正向传播
    def forward(self, x):
        for i, linear in enumerate(self.linears):
            x = linear(x)
        
            print(f"layer:{i},std:{x.std()}")

            if torch.isnan(x.std()):
                print(f"output is nan in {i} layers")
                break
        
        return x
    
    # 权值初始化，使用标准正态分布
    def initialize(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                # 改变初始化std
                # nn.init.normal_(m.weight.data)
                nn.init.normal_(m.weight.data, std=np.sqrt(1/self.neural_num))

In [5]:
layer_nums = 100
neural_nums = 256
batch_size = 16

net = MLP(neural_nums, layer_nums)
net.initialize()

inputs = torch.randn((batch_size, neural_nums))

output = net(inputs)
output
# 可以看到结果不会出现nan的情况了

layer:0,std:1.0278830528259277
layer:1,std:1.0181204080581665
layer:2,std:1.014767050743103
layer:3,std:1.0146775245666504
layer:4,std:1.0123896598815918
layer:5,std:1.0262004137039185
layer:6,std:1.0095330476760864
layer:7,std:1.027438759803772
layer:8,std:1.019274115562439
layer:9,std:1.0268949270248413
layer:10,std:1.0337014198303223
layer:11,std:1.0398966073989868
layer:12,std:1.04704749584198
layer:13,std:1.072679042816162
layer:14,std:1.0385937690734863
layer:15,std:1.0226327180862427
layer:16,std:1.0111712217330933
layer:17,std:1.0268144607543945
layer:18,std:1.0496608018875122
layer:19,std:1.0328172445297241
layer:20,std:1.0183610916137695
layer:21,std:1.0026816129684448
layer:22,std:0.9916945099830627
layer:23,std:1.0056698322296143
layer:24,std:1.0347695350646973
layer:25,std:1.0429946184158325
layer:26,std:1.071308970451355
layer:27,std:1.0872653722763062
layer:28,std:1.0766782760620117
layer:29,std:1.035771369934082
layer:30,std:1.0315487384796143
layer:31,std:1.02746701240

tensor([[ 1.9680,  4.9377,  0.5148,  ..., -0.0078, -2.6069, -0.9856],
        [-0.5244, -0.2836,  0.5604,  ...,  0.3125,  0.6122,  0.4675],
        [ 0.7246,  1.0515,  0.3562,  ...,  0.1150, -0.6337, -0.5806],
        ...,
        [-0.8396,  0.0318, -0.2757,  ...,  0.1789, -0.0826, -0.3895],
        [ 0.2004,  1.5627,  0.3905,  ...,  0.3986, -0.7640,  0.3204],
        [-0.1645, -2.7014, -0.2713,  ..., -0.1793,  1.4589,  0.7029]],
       grad_fn=<MmBackward0>)

**采用恰当的权值初始化方法，可以实现多层神经网络的输出值尺度维持在一定范围内，这样反向传播的时候，有利于缓解梯度消失或爆炸**

当然上面的网络只是线性网络，当存在激活函数时，方差会越来越小，也可能会发生梯度消失现象，这就需要其他的初始化方法。

<img style="float: center;" src="images/80.png" width="70%">

## Xavier初始化

**方差一致性：**保持数据尺度维持在恰当的范围，通常方差为1

2010年，Xavier发表一篇文章《Understanding the difficulty of training deep feedforward neural networks》，详细探讨有激活函数时，如何进行权重初始化（运用方差一致性原则），但是这里考虑的是**饱和激活函数**（sigmoid，tanh）。

推导公式如下：

$n_i * D(W) = 1$

$n_{i+1} * D(W) = 1$

$\Rightarrow D(W) = \frac{2}{n_i+n_{i+1}}$

这里$n_i, n_{i+1}$分别代表输入层和输出层的神经元个数。

Xavier采用均匀分布对权重进行初始化，均匀分布上下限：

$W\sim U[-a, a]$

$D(W)=\frac{(-a-a)^2}{12}=\frac{(2a)^2}{12}=\frac{a^2}{3}$

综合上面两个$D(W)$公式，可得：

$\frac{2}{n_i+n_{i+1}}=\frac{a^2}{3} \Rightarrow a=\frac{\sqrt{6}}{\sqrt{n_i+n{i+1}}}$

$W \sim U\left[-\frac{\sqrt{6}}{\sqrt{n_i+n{i+1}}}, \frac{\sqrt{6}}{\sqrt{n_i+n{i+1}}}\right]$

In [6]:
class MLP(nn.Module):
    def __init__(self, neural_num, layers):
        super(MLP, self).__init__()
        self.linears = nn.ModuleList([nn.Linear(neural_num, neural_num, bias=False)
                                     for i in range(layers)])
        self.neural_num = neural_num
    
    # 正向传播
    def forward(self, x):
        for i, linear in enumerate(self.linears):
            x = linear(x)
            x = torch.tanh(x)
        
            print(f"layer:{i},std:{x.std()}")

            if torch.isnan(x.std()):
                print(f"output is nan in {i} layers")
                break
        
        return x
    
    # 权值初始化，使用标准正态分布
    def initialize(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                # Xavier初始化权重
                # nn.init.calculate_gain(nonlinearity, param=None)
                # nonlinearity: 激活函数名称，如tanh
                # param: 激活函数参数，如Leaky ReLU的negative_slop
                # 计算激活函数的方差变化尺度
                # 方差变化尺度=输入数据方差/经过激活函数后输出数据方差
                tanh_gain = nn.init.calculate_gain('tanh')
                nn.init.xavier_uniform_(m.weight.data, gain=tanh_gain)

In [7]:
layer_nums = 100
neural_nums = 256
batch_size = 16

net = MLP(neural_nums, layer_nums)
net.initialize()

inputs = torch.randn((batch_size, neural_nums))

output = net(inputs)
output

layer:0,std:0.7597724795341492
layer:1,std:0.691631019115448
layer:2,std:0.6674433350563049
layer:3,std:0.6625684499740601
layer:4,std:0.6591121554374695
layer:5,std:0.6541376113891602
layer:6,std:0.657918393611908
layer:7,std:0.6530606746673584
layer:8,std:0.6525726914405823
layer:9,std:0.6433029770851135
layer:10,std:0.6500066518783569
layer:11,std:0.6507823467254639
layer:12,std:0.6547849774360657
layer:13,std:0.6571223735809326
layer:14,std:0.659264862537384
layer:15,std:0.6672247648239136
layer:16,std:0.6492877006530762
layer:17,std:0.6394469738006592
layer:18,std:0.6479735374450684
layer:19,std:0.6470638513565063
layer:20,std:0.6510664224624634
layer:21,std:0.6546648740768433
layer:22,std:0.65529865026474
layer:23,std:0.6478891968727112
layer:24,std:0.6474164724349976
layer:25,std:0.6427276730537415
layer:26,std:0.6494070887565613
layer:27,std:0.6561998128890991
layer:28,std:0.6532543301582336
layer:29,std:0.661830484867096
layer:30,std:0.6557857394218445
layer:31,std:0.653790473

tensor([[ 0.1929, -0.9175,  0.6204,  ...,  0.7386,  0.1075, -0.3891],
        [-0.3442,  0.8802,  0.1398,  ..., -0.0068, -0.4867, -0.9939],
        [-0.5447, -0.8563, -0.9602,  ...,  0.4901,  0.7336,  0.7447],
        ...,
        [ 0.0184, -0.7887, -0.6479,  ...,  0.7249,  0.4065, -0.9407],
        [-0.7349,  0.9036,  0.7570,  ..., -0.7527,  0.1232,  0.7407],
        [ 0.2886,  0.6682, -0.8228,  ...,  0.8385, -0.5707, -0.5666]],
       grad_fn=<TanhBackward0>)

2012年AlexNet出现后，非饱和函数ReLU也用到了神经网络中，然而Xavier初始化对ReLU失效。

In [8]:
class MLP(nn.Module):
    def __init__(self, neural_num, layers):
        super(MLP, self).__init__()
        self.linears = nn.ModuleList([nn.Linear(neural_num, neural_num, bias=False)
                                     for i in range(layers)])
        self.neural_num = neural_num
    
    # 正向传播
    def forward(self, x):
        for i, linear in enumerate(self.linears):
            x = linear(x)
            x = torch.relu(x)
        
            print(f"layer:{i},std:{x.std()}")

            if torch.isnan(x.std()):
                print(f"output is nan in {i} layers")
                break
        
        return x
    
    # 权值初始化，使用标准正态分布
    def initialize(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                # Xavier初始化权重
                # nn.init.calculate_gain(nonlinearity, param=None)
                # nonlinearity: 激活函数名称，如tanh
                # param: 激活函数参数，如Leaky ReLU的negative_slop
                # 计算激活函数的方差变化尺度
                # 方差变化尺度=输入数据方差/经过激活函数后输出数据方差
                relu_gain = nn.init.calculate_gain('relu')
                nn.init.xavier_uniform_(m.weight.data, gain=relu_gain)

In [9]:
layer_nums = 100
neural_nums = 256
batch_size = 16

net = MLP(neural_nums, layer_nums)
net.initialize()

inputs = torch.randn((batch_size, neural_nums))

output = net(inputs)
output

layer:0,std:0.8272078633308411
layer:1,std:0.8287857174873352
layer:2,std:0.8225235342979431
layer:3,std:0.8669309020042419
layer:4,std:0.8629841208457947
layer:5,std:0.9074069857597351
layer:6,std:0.8765947222709656
layer:7,std:0.8703077435493469
layer:8,std:0.8116634488105774
layer:9,std:0.8748800158500671
layer:10,std:0.9287486672401428
layer:11,std:1.0342388153076172
layer:12,std:1.1486414670944214
layer:13,std:1.139365553855896
layer:14,std:1.1594090461730957
layer:15,std:1.2147501707077026
layer:16,std:1.166542410850525
layer:17,std:1.162285327911377
layer:18,std:1.1240496635437012
layer:19,std:1.068245768547058
layer:20,std:1.2006644010543823
layer:21,std:1.240335464477539
layer:22,std:1.4015294313430786
layer:23,std:1.388387680053711
layer:24,std:1.5033951997756958
layer:25,std:1.641709804534912
layer:26,std:1.3320049047470093
layer:27,std:1.3643521070480347
layer:28,std:1.3563874959945679
layer:29,std:1.3409533500671387
layer:30,std:1.400130271911621
layer:31,std:1.50351476669

tensor([[0.0000, 1.3819, 0.3579,  ..., 3.7246, 1.9951, 3.3243],
        [0.0000, 1.3031, 0.2963,  ..., 3.7317, 1.8890, 3.2710],
        [0.0000, 0.8254, 0.2258,  ..., 2.1510, 1.1945, 1.9262],
        ...,
        [0.0000, 0.9481, 0.2314,  ..., 2.2969, 1.3180, 2.1258],
        [0.0000, 0.9808, 0.2747,  ..., 2.4863, 1.3197, 2.1989],
        [0.0000, 0.6398, 0.1703,  ..., 1.6777, 0.8792, 1.4625]],
       grad_fn=<ReluBackward0>)

## Kaiming初始化

遵循：方差一致性原则

针对：ReLU激活函数及其变种

最后权重标准差：

$D(W)=\frac{2}{n_i}$

$D(W)=\frac{2}{\left(1+a^2\right)*n_i}$

$D(W)=\sqrt{\frac{2}{\left(1+a^2\right)*n_i}}$

In [10]:
class MLP(nn.Module):
    def __init__(self, neural_num, layers):
        super(MLP, self).__init__()
        self.linears = nn.ModuleList([nn.Linear(neural_num, neural_num, bias=False)
                                     for i in range(layers)])
        self.neural_num = neural_num
    
    # 正向传播
    def forward(self, x):
        for i, linear in enumerate(self.linears):
            x = linear(x)
            x = torch.relu(x)
        
            print(f"layer:{i},std:{x.std()}")

            if torch.isnan(x.std()):
                print(f"output is nan in {i} layers")
                break
        
        return x
    
    # 权值初始化，使用标准正态分布
    def initialize(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight.data)

In [11]:
layer_nums = 100
neural_nums = 256
batch_size = 16

net = MLP(neural_nums, layer_nums)
net.initialize()

inputs = torch.randn((batch_size, neural_nums))

output = net(inputs)
output

layer:0,std:0.8194370865821838
layer:1,std:0.7863913774490356
layer:2,std:0.7790502905845642
layer:3,std:0.7336016297340393
layer:4,std:0.721562385559082
layer:5,std:0.719891369342804
layer:6,std:0.7007282376289368
layer:7,std:0.63581383228302
layer:8,std:0.6558110117912292
layer:9,std:0.6231154799461365
layer:10,std:0.6212367415428162
layer:11,std:0.5692794322967529
layer:12,std:0.5806706547737122
layer:13,std:0.5602917671203613
layer:14,std:0.5692883729934692
layer:15,std:0.5992907881736755
layer:16,std:0.5045601725578308
layer:17,std:0.5361236333847046
layer:18,std:0.545310378074646
layer:19,std:0.487845242023468
layer:20,std:0.44877204298973083
layer:21,std:0.41723302006721497
layer:22,std:0.4157910645008087
layer:23,std:0.42320042848587036
layer:24,std:0.47227609157562256
layer:25,std:0.4711170196533203
layer:26,std:0.47471195459365845
layer:27,std:0.4232582449913025
layer:28,std:0.38460591435432434
layer:29,std:0.38494592905044556
layer:30,std:0.4043668508529663
layer:31,std:0.37

tensor([[0.0000, 1.8185, 0.4060,  ..., 1.9074, 0.4009, 1.4617],
        [0.0000, 1.2005, 0.3790,  ..., 1.2151, 0.4098, 0.9926],
        [0.0000, 1.7457, 0.4749,  ..., 1.8213, 0.4690, 1.4190],
        ...,
        [0.0000, 2.0995, 0.5222,  ..., 2.2139, 0.6825, 1.7255],
        [0.0000, 0.8958, 0.3022,  ..., 0.9257, 0.3139, 0.7616],
        [0.0000, 1.2969, 0.3834,  ..., 1.2798, 0.4639, 1.0696]],
       grad_fn=<ReluBackward0>)

## 十种权重初始化方法

PyTorch提供4大类权重初始化方法：
- 针对饱和激活函数（sigmoid，tanh）：Xavier均匀分布，Xavier正态分布
- 针对非饱和激活函数（relu及其变种）：Kaiming均匀分布，Kaiming正态分布
- 常用的分布初始化方法：均匀分布，正态分布，常数分布
- 特殊的矩阵初始化方法：正交矩阵初始化，单位矩阵初始化，稀疏矩阵初始化

# 损失函数

## 简介

损失函数：衡量模型与真实标签的差异。

- 损失函数Loss Function：计算一个样本的一个差异。$Loss=f(\hat{y},y)$
- 代价函数Cost Function：计算整个训练集Loss的平均值。$Cost=\frac{1}{N}\sum^N_i f(\hat{y}, y)$
- 目标函数Objective Function：模型训练的最终目标，过拟合与欠拟合之间权衡。$Obj=Cost+Regularization$

一般在衡量模型输出和真实标签差异时，往往用损失函数。

PyTorch中损失函数：
<img style="float: center;" src="images/81.png" width="70%">

\_Loss也是继承Module，因此也包含8个参数字典，同时需要设置一个reduction参数。

以人民币二分类为例：

先在定义损失函数和使用损失函数的地方打上断点：
<img style="float: center;" src="images/82.png" width="70%">

**损失函数定义过程**

程序运行到第一个断点处，进入loss.py文件中的一个class CrossEntropyLoss(\_WeightedLoss)：交叉熵损失类的\_\_init\_\_方法，可以发现交叉熵损失函数继承\_WeightedLoss类：
<img style="float: center;" src="images/83.png" width="70%">

继续步入，进入class \_WeightedLoss(\_Loss)类中，继承\_Loss类，继续步入\_Loss类，继承Module，因此损失函数的初始化方法和模型类似，也是调用Module的初始化方法，最终会有8个属性字典，然后设置reduction参数。

**损失函数使用过程**

进入第二个断点，步入，由于损失函数也是一个Module，因此调用时也是调用forward方法：
<img style="float: center;" src="images/84.png" width="70%">

再次步入，可以看到损失函数中的forward样子，在模型构建里的forward写的是各个模块的拼接方式，而损失函数的forward里调用F里的各种函数，进入该函数，可以看到交叉熵损失函数：
<img style="float: center;" src="images/85.png" width="70%">
<img style="float: center;" src="images/86.png" width="70%">

以上就是损失函数初始化和使用方法的内部运行机制。

损失函数其实也是一个Module，初始化依然有8个属性字典，使用方法定义在forward函数中。

## 交叉熵损失CrossEntropyLoss

nn.CrossEntropyLoss：nn.LogSortmax()与nn.NLLLoss()结合，进行交叉熵计算。

nn.CrossEntropyLoss(weight=None, size_average=None, ignore_index=-100, reduce=None, reduction='mean')

- weight：各类别loss设置权重
- ignore_index：忽略某个类别
- reduction：计算模式
  - none：逐元素计算，多少个样本返回多少个loss
  - sum：所有元素loss求和，返回标量
  - mean：所有元素loss求加权平均，返回标量

交叉熵损失函数：$H(P,Q)=-\sum^N_{i=1}P(x_i)\log Q(x_i)$
- P表示数据的原始分布
- Q表示模型的输出分布

交叉熵损失衡量两个分布之间的差异程度，交叉熵越低，说明两个分布越接近。

PyTorch中交叉熵损失先用nn.LogSoftmax()把模型的输出值归一化成概率分布形式，然后用单个样本输出，并且没有求和符号。

为什么交叉熵损失能衡量两个分布之间的差异？

熵：信息论之父香农从热力学借鉴来的名词，用于描述事件的不确定性，事物的不确定性越大，则熵就越大。（例如，明天会下雨的熵就比明天太阳从东边升起的熵要大）

熵的公式：$H(P)=E_{x\sim p}[I(x)]=-\sum^N_{i=0}P(x_i)\log P(x_i)$

熵是自信息的期望，自信息：$I(x)=-\log p(x)$，一个事件发生的概率，然后取对数再取反，即一个事件发生的概率越大，则自信息越少。所有事件发生的概率都很大，则熵就会小，事件的不确定性就小。

下图是一个两点分布的信息熵，当概率是0.5时熵最大，即事件的不确定性最大，熵大约是0.69（二分类模型中，模型训练坏了，或者刚训练时，Loss的值可能为0.69，这时候就说模型目前没有任何判断能力）
<img style="float: center;" src="images/87.png" width="70%">

**相对熵**又称KL散度，用于衡量两个分布之间的差异（距离），但不是一个距离函数（因为距离函数具有对称性，即p到q的距离等于q到p的距离）：

$D_{KL}(P,Q)$

$=E_{x\sim p}\left[\log\frac{P(x)}{Q(x)}\right]$

$=E_{x\sim p}\left[\log{P(x_i)}-\log{Q(x_i)}\right]$

$=\sum^N_{i=1}P(x_i)\left[\log{P(x_i)}-\log{Q(x_i)}\right]$

$=\sum^N_{i=1}P(x_i)\log{P(x_i)}-\sum^N_{i=1}P(x_i)\log{Q(x_i)}$

这里$P$是数据的真实分布，$Q$是模型输出的分布，此处就是用$Q$的分布去逼近$P$的分布。（因此不具备对称性）

**交叉熵**：交叉熵=信息熵+相对熵

$H(P,Q)=-\sum^N_{i=1}P(x_i)\log Q(x_i)$

即，$H(P,Q)=D_{KL}(P,Q)+H(P)$

机器学习中，最小化交叉熵，就是最小化相对熵，因为训练集取出来之后，信息熵就固定了。

softmax可以将一个输出值转换到概率取值的一个范围：

$loss(x, class)=-\log\left(\frac{\exp(x[class])}{\sum_j \exp(x[j])}
\right)=-x[class]+\log\left(\sum_j\exp(x[j])\right)$

此处x为输出概率值，class为某一类别，括号里执行一个softmax，把某个神经元的输出归一化成概率取值，然后$-\log$一下，就得到交叉熵损失函数。

对比交叉熵公式：$H(P,Q)=-\sum^N_{i=1}P(x_i)\log Q(x_i)$

由于是某个样本，则$P(x_i)$是1（因为已经取出来了），而是某个样本，也不用求和符号。这就是用softmax的原因，把模型的输出值转成概率分布形式，得到交叉熵损失函数。

之后说一下参数：

nn.CrossEntropyLoss(weight=None, size_average=None, ignore_index=-100, reduce=None, reduction='mean')
- weight：各类别loss设置权值，如果类别不均衡则这个参数就很重要：
  - $loss(x, class)=weight[class]\left((-x[class]+\log\left(\sum_j\exp(x[j])\right)\right)$
  - 如果想让模型更关注某一个类，可以把这一类的权值设置的更大一点。
- ignore_index：表示某个类别不计算loss
- reduction：3个计算模式none/sum/mean

In [12]:
# 模型预测的输出
# 两个类，可以看到模型输出是数值，得softmax转成分布
inputs = torch.tensor([[1, 2], [1, 3], [1, 3]], dtype=torch.float) 
# 这里的类型必须是long，两个类0和1
target = torch.tensor([0, 1, 1], dtype=torch.long)

# 三种模式的损失函数
loss_f_none = nn.CrossEntropyLoss(weight=None, reduction='none')
loss_f_sum = nn.CrossEntropyLoss(weight=None, reduction='sum')
loss_f_mean = nn.CrossEntropyLoss(weight=None, reduction='mean')

# forward
loss_none = loss_f_none(inputs, target)
loss_sum = loss_f_sum(inputs, target)
loss_mean = loss_f_mean(inputs, target)

# view
print("Cross Entropy Loss:\n ", loss_none, loss_sum, loss_mean)

Cross Entropy Loss:
  tensor([1.3133, 0.1269, 0.1269]) tensor(1.5671) tensor(0.5224)


weight损失，给类别加上权值后，对应样本的损失就会相应加倍

关于mean模式loss计算：三个样本，第一个权值1，后两个权值2，分母变为1+2+2=5，所以mean模式下求平均不是除以样本的个数，而是样本所占权值的总份数。
<img style="float: center;" src="images/88.png" width="70%">

### 交叉熵损失函数的特例

#### nn.NLLLoss

上面的交叉熵损失中，发现是softmax和NLLLoss组合。

此处nn.NLLLoss其实只是实现了一个负号功能

nn.NLLLoss：实现负对数似然函数里的负号功能

nn.NLLLoss(weight=None, size_average=None, ignore_index=-100, reduce=None, reduction='mean')
<img style="float: center;" src="images/89.png" width="70%">

这个损失函数根据真实类别去获取相应的softmax之后的概率结果，然后取反得到最终损失。

#### nn.BCELoss

二分类交叉熵：输入取值[0,1]

nn.BCELoss(weight=None, size_average=None, reduce=None, reduction='mean')

计算公式：$l_n=-w_n\left[y_n*\log x_n+(1-y_n)*\log(1-x_n)\right]$
<img style="float: center;" src="images/90.png" width="70%">

注意这里target的类型是float，每个样本属于哪一类需要写成独热编码（每个神经元一一对应去计算loss，而不是一整个神经元向量去计算loss）

#### nn.BCEWithLogitsLoss

该函数结合Sigmoid和二分类交叉熵（注意，网络最后不加sigmoid函数）

nn.BCEWithLogitsLoss(weight=None, size_average=None, reduce=None, reduction='mean', pos_weight=None)

这里参数多了一个pow_weight，主要用于平衡正负样本，对正样本进行一个权值设定。（例如正样本有100个，负样本有300个，则该数可以设为3）

计算公式：$l_n=-w_n\left[y_n*\log\sigma(x_n)+(1-y_n)*\log(1-\sigma(x_n))\right]$

## 14种损失函数

- 分类问题
  - 二分类单标签问题：nn.BCELoss, nn.BCEWithLogitsLoss, nn.SoftMarginLoss
  - 二分类多标签问题：nn.MultiLabelSoftMarginLoss
  - 多分类单标签问题: nn.CrossEntropyLoss, nn.NLLLoss, nn.MultiMarginLoss
  - 多分类多标签问题: nn.MultiLabelMarginLoss,
  - 不常用：nn.PoissonNLLLoss, nn.KLDivLoss
- 回归问题: nn.L1Loss, nn.MSELoss, nn.SmoothL1Loss
- 时序问题：nn.CTCLoss
- 人脸识别问题：nn.TripletMarginLoss
- 半监督Embedding问题(输入之间的相似性): nn.MarginRankingLoss, nn.HingeEmbeddingLoss, nn.CosineEmbeddingLoss

### nn.L1Loss

用于回归问题，计算inputs与target之差的绝对值

nn.L1Loss(size_average=None, reduce=None, reduction='mean')

计算公式：$l_n=|x_n-y_n|$

### nn.MSE

用于回归问题，计算inputs与target之差的平方

nn.MSELoss(size_average=None, reduce=None, reduction='mean')

计算公式：$l_n=(x_n-y_n)^2$

### nn.SmoothL1Loss

平滑的L1Loss（回归问题）

nn.SmoothL1Loss(size_average=None, reduce=None, reduction='mean')

计算公式：$loss(x,y)=\frac{1}{n}\sum_iz_i$

$
z_i=
\left\{ 
\begin{aligned}
0.5(x_i-y_i)^2, |x_i-y_i|\lt 1\\ 
|x_i-y_i|-0.5, |x_i-y_i|\ge 1
\end{aligned}
\right.
$

采用这种平滑损失函数可以减轻离群点带来的影响
<img style="float: center;" src="images/91.png" width="70%">

### nn.PoissonNLLLoss

泊松分布的负对数似然损失函数（分类里如果发现数据的类别服从泊松分布，可以用这个损失函数）

nn.PoissonNLLLoss(log_input=True, full=False, size_average=None, eps=1e-8, reduce=None, reduction='mean')
- log_input：输入是否为对数形式
  - 若为True：$loss(input,target)=\exp(input)-target\times input$
  - 若为False：$loss(input,target)=input-target\times \log(input+eps)$
- full：计算所有loss，默认为False
- eps：修正项，避免log(input)为nan

### nn.KLDivLoss

计算KLD，KL散度，相对熵。

**注意：需要提前将输入计算log-probabilities，例如通过nn.logsoftmax**

nn.KLDivLoss(size_average=None, reduce=None, reduction='mean')

<img style="float: center;" src="images/92.png" width="70%">

上面PyTorch里的计算与我们原来公式里的计算有点不一样，需要先logsoftmax()，完成转换为分布然后转成对数才可以。

此处reduction还多了一种计算模式'batchmean'，按照batchsize大小求平均值。

### nn.MarginRankingLoss

计算两个向量之间的相似度，用于排序任务。

该方法计算两组数据之间的差异，即每个元素两两之间都会计算差异，返回一个n\*n的loss矩阵，类似于相关性矩阵。

nn.MarginRankingLoss(margin=0.0, size_average=None, reduce=None, reduction='mean')

margin表示边界值，x1与x2之间的差异值。

计算公式：$loss(x,y)=\max(0, -y*(x_1-x_2)+margin)$
- y=1时，希望x1比x2大，当x1>x2时，不产生loss
- y=-1时，希望x2比x1大，当x2>x1时，不产生loss

<img style="float: center;" src="images/93.png" width="70%">

### nn.MultiLabelMarginLoss

多标签边界损失函数，多标签分类，一个样本可能属于多个标签，与多分类任务不一样（多标签问题）。

nn.MultiLabelMarginLoss(size_average=None, reduce=None, reduction='mean')

计算公式：$loss(x,y)=\sum_{i,j}\frac{max(0, 1-(x[y[j]]-x[i]))}{x.size(0)}$
，
此处i的取值从0到输出的维度减1，j的取值也是0到输出的维度减1，对于所有的i和j，i不等于y[j]，即标签所在神经元减去那些非标签所在神经元。
<img style="float: center;" src="images/94.png" width="70%">

假设有一个训练样本，输出层4个神经元（4分类问题），前向传播后，神经网络的四个神经元输出分布是[0.1, 0.2, 0.4, 0.8]，而这个样本的真实标签是[0, 3, -1, -1]，这说明该样本属于第0类和第3类（必须为torch.long型），并且必须和输出神经元个数一样，属于哪几类写再前面，不足用-1填补。

使用多标签边界损失函数时，具体计算如下：

输入样本属于0和3这两类，不属于1和2，根据上述公式，后面那部分是标签所在的神经元减去标签不在的神经元，比如标签在第0个神经元：
- item_1=(1-(x[0]-x[1]))+(1-(x[0]-x[2]))  # 标签在第0个神经元
- item_2=(1-(x[3]-x[1]))+(1-(x[3]-x[2]))  # 标签在第3个神经元
- 两部分损失相加除以总的神经元个数：loss=(item_1+item_3)/x.shape[0]

希望标签所在神经元要比非标签所在神经元的输出值要尽量的大：
- 当这个差大于1时，根据max(0, 1-差值)才发现不会有损失产生
- 当这个差小或非标签所在的神经元比标签所在神经元大的时候，就会产生损失

因此，上面那个例子，要让第0个神经元的值比第1/2个大一些，第3个神经元的值比第1/2个大一些，才能说明这个样本属于第0类和第3类。

### nn.SoftMarginLoss

计算二分类的logistic损失（二分类问题）

nn.SoftMarginLoss(size_average=None, reduce=None, reduction='mean')

计算公式：$loss(x,y)=\sum_i\frac{\log(1+\exp(-y[i]*x[i]))}{x.nelement()}$

### nn.MultiLabelSortMarginLoss

SoftMarginLoss多标签版本（多标签问题）

nn.MultiLabelSoftMarginLoss(weight=None, size_average=None, reduce=None, reduction='mean')

weight表示各类别的loss设置权值

计算公式：$loss(x,y)=-\frac{1}{C}*\sum_i y[i] * \log\left((1+\exp(-x[i]))^{-1}\right)+(1-y[i])*\log\left(\frac{\exp(-x[i])}{(1+\exp(-x[i]))})\right)$

以三分类任务为例，输入的这个样本属于第二类和第三类：
<img style="float: center;" src="images/95.png" width="70%">

### nn.MultiMarginLoss(hingLoss)

计算多分类的折页损失（多分类问题）

nn.MultiMarginLoss(p=1, margin=1.0, weight=None, size_average=None, reduce=None, reduction='mean')

此处p可选1或2，margin表示边界值

计算公式：$loss(x,y)=\frac{\sum_i\max(0, margin-x[y]+x[i])^p}{x.size(0)}$

此处x，y是[0, 神经元个数-1]，对于所有i和j，i不等于y[j]。

这里就类似于hing loss，x[y]表示标签所在的神经元，x[i]表示非标签所在神经元。
<img style="float: center;" src="images/96.png" width="70%">

这个其实与多标签边界损失函数原理差不多，只不过那里是一个样本属于多个类，需要每个类都这样计算，而这里一个样本属于1个类，只需要计算一次即可。
<img style="float: center;" src="images/97.png" width="70%">

假设现在有3个类别，得分函数计算某张图片的得分为$f(x_i,w)=[13, -7, 11]$，而实际结果是第一类($y_i=0$)，假设$\triangle=10$，这个是margin，则上面的公式就把错误类别($j\ne y_i$)都遍历了一遍，求值加和：$L_i=\max(0, -7-13+10)+\max(0, 11-13+10)$

这个损失和交叉熵损失是不同的两种评判标准：
- 多分类折页损失聚焦于分类错误的与正确类别之间的惩罚距离越小越好
- 交叉熵损失聚焦分类正确的概率分布越大越好

### nn.TripletMarginLoss

计算三元组损失，人脸验证中常用

nn.TripleMarginLoss(margin=1.0, p=2.0, eps=1e-6, swap=False, size_average=None, reduce=None, reduction='mean')

此处p表示范数的阶。

计算公式：$L(a,p,n)=\max(d(a_i,p_i)-d(a_i,n_i)+margin, 0)$

在人脸识别训练模型时，往往需要把训练集做成三元组(A, P, N)，A和P是同一个人，A和N不是同一个人，然后训练我们的模型

让模型把A和P看成一样，争取让A和P之间的距离小，而A和N之间的距离大，则模型就可以进行人脸识别任务了。

### nn.HingeEmbeddingLoss

计算两个输入的相似性，常用于非线性embedding和半监督学习

注意：输入的x应为两个输入之差的绝对值，即手动计算两个输入的差值

nn.HingeEmbeddingLoss(margin=1.0, size_average=None, reduce=None, reduction='mean')

计算公式：
$
l_n=
\left\{ 
\begin{aligned}
x_n, y_n = 1\\ 
\max(0, \triangle - x_n), y_n=-1
\end{aligned}
\right.
$

### nn.CosineEmbeddingLoss

采用余弦相似度计算两个输入的相似性，常用于半监督学习和embedding

nn.CosineEmbeddingLoss(margin=0.0, size_average=None, reduce=None, reduction='mean')

此处margin可取值[-1,1]，推荐为[0,0.5]，计算公式：

$
loss(x,y)=
\left\{ 
\begin{aligned}
1-\cos(x_1,x_2), y_n = 1\\ 
\max(0, \cos(x_1,x_2)-margin), y_n=-1
\end{aligned}
\right.
$

之所以用cos，希望关注于这两个输入方向上的一个差异，而不是距离上的差异，cos函数如下：

$\cos(\theta)=\frac{A\cdot B}{\|A\|\|B\|}=\frac{\sum^n_{i=1}A_i\times B_i}{\sqrt{\sum^n_{i=1}(A_i)^2}\times \sqrt{\sum^n_{i=1}(B_i)^2}}$

### nn.CTCLoss

计算CTC损失，解决时序类数据的分类

nn.CTCLoss(blank=0, reduction='mean', zero_infinity=False)
- blank：空标签
- zero_infinity：无穷大的值或者梯度置0

# 思维导图

<img style="float: center;" src="images/98.png" width="70%">